In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
f1 = pd.read_excel('./data/(input)rawdata.xlsx')
f1.head()

,group,ICD-10,ICD-10 in OSCC I,MMP-1_Conc.
0,2,NaN,NaN,0.0
1,2,NaN,NaN,0.0
2,2,NaN,NaN,0.0
3,2,NaN,NaN,0.0
4,2,NaN,NaN,0.0


In [3]:
def SampleSizeEstimator_Sensitivity(P_H0, P_H1, power=0.8, alpha=0.05):
    p0= P_H0
    p1 = P_H1
    z0 = st.norm.ppf(1-alpha/2)
    z1 = st.norm.ppf(power)
    n = 1
    while True:
        right = (1/(2*(n+z0**2)))*((2*n*p0+z0**2+1)+z0*np.sqrt(z0**2+(2+1/n)+4*p0*(n*(1-p0)-1)))
        left = (1/(2*(n+z1**2)))*((2*n*p1+z1**2-1)-z1*np.sqrt(z1**2-(2+1/n)+4*p1*(n*(1-p1)+1)))
        if right > left:
            n += 1
        else:
            break
    return n

In [4]:
OSCC = 270
OSCC_OPMD = OSCC + 603
cutoff =250
power = 0.8
alpha = 0.05


f2 = open('./results/under_the_sensitivity_assumption.csv','w')
f2.write('site,se0,se1,power,alpha,prev_inOSCC,prev_inOSCC+OPMD,n_site,n_OSCC,n_OSCC+OPMD\n')

for site in ['A','B','C','ABC']:
    if site !='ABC':
        D = f1[(f1['group']==3)&(f1['ICD-10 in OSCC I']==site)]['MMP-1_Conc.']
    elif site == 'ABC':
        D = f1[(f1['group']==3)]['MMP-1_Conc.']
    p = len(D[D>cutoff])/len(D)
    prev_inOSCC = len(D)/OSCC
    prev_inOSCC_OPMD = len(D)/OSCC_OPMD
    
    for ph1 in [0.75, 0.8, 0.85, 0.9,p]:
        for ph0 in [0.7, 0.75, 0.8, 0.85]:
            if ph1 > ph0:
                n = SampleSizeEstimator_Sensitivity(ph0,ph1,power=power,alpha=alpha)
                n_OSCC = int(np.ceil(n/prev_inOSCC))
                n_OSCC_OPMD = int(np.ceil(n/prev_inOSCC_OPMD))
                f2.write('%s,%s,%s,%s,%s,%.2f%%,%.2f%%,%d,%d,%d\n'%(site,ph0,ph1,power,alpha,prev_inOSCC*100,prev_inOSCC_OPMD*100,n,n_OSCC,n_OSCC_OPMD))
            else:
                continue
f2.close()

In [5]:
def SampleSizeEstimator_MarginError(P_H1,marginErr, power=0.8, alpha=0.05):
    z = st.norm.ppf(1-alpha/2)
    p = P_H1
    d = marginErr
    n = (z**2*(p*(1-p)))/d**2
    return int(np.ceil(n))

In [6]:
OSCC = 270
OSCC_OPMD = OSCC + 603
cutoff =250
power = 0.8
alpha = 0.05


f2 = open('./results/under_the_sensitivity_assumption_with_marginal_error.csv','w')
f2.write('site,margin,se1,power,alpha,prev_inOSCC,prev_inOSCC+OPMD,n_site,n_OSCC,n_OSCC+OPMD\n')

for site in ['A','B','C','ABC']:
    if site !='ABC':
        D = f1[(f1['group']==3)&(f1['ICD-10 in OSCC I']==site)]['MMP-1_Conc.']
    elif site == 'ABC':
        D = f1[(f1['group']==3)]['MMP-1_Conc.']
    p = len(D[D>cutoff])/len(D)
    prev_inOSCC = len(D)/OSCC
    prev_inOSCC_OPMD = len(D)/OSCC_OPMD
    
    
    for ph1 in [0.75, 0.8, 0.85, 0.9,p]:
        if p >0.8:
            marginerror = p-0.8
        elif p > 0.75:
            marginerror = p-0.75
        elif p > 0.7:
            marginerror = p-0.7
        if ph1 == p:
            for margin in [0.05,0.04,0.03,0.02,0.01,marginerror]:
                n = SampleSizeEstimator_MarginError(ph1,margin,power=power,alpha=alpha)
                n_OSCC = int(np.ceil(n/prev_inOSCC))
                n_OSCC_OPMD = int(np.ceil(n/prev_inOSCC_OPMD))
                f2.write('%s,%s,%s,%s,%s,%.2f%%,%.2f%%,%d,%d,%d\n'%(site,margin,ph1,power,alpha,prev_inOSCC*100,prev_inOSCC_OPMD*100,n,n_OSCC,n_OSCC_OPMD))
        else:    
            for margin in [0.05,0.04,0.03,0.02,0.01]:
                n = SampleSizeEstimator_MarginError(ph1,margin,power=power,alpha=alpha)
                n_OSCC = int(np.ceil(n/prev_inOSCC))
                n_OSCC_OPMD = int(np.ceil(n/prev_inOSCC_OPMD))
                f2.write('%s,%s,%s,%s,%s,%.2f%%,%.2f%%,%d,%d,%d\n'%(site,margin,ph1,power,alpha,prev_inOSCC*100,prev_inOSCC_OPMD*100,n,n_OSCC,n_OSCC_OPMD))
            
f2.close()